In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress

#import csv
cta_df = pd.read_csv("CTA_Ridership.csv")
cta_df

,service_date,day_type,bus,rail_boardings,total_rides
0,01/01/2001,U,297192,126455,423647
1,01/02/2001,W,780827,501952,1282779
2,01/03/2001,W,824923,536432,1361355
3,01/04/2001,W,870021,550011,1420032
4,01/05/2001,W,890426,557917,1448343
...,...,...,...,...,...
7331,11/26/2020,U,119470,56839,176309
7332,11/27/2020,W,221878,106692,328570
7333,11/28/2020,A,198860,97141,296001
7334,11/29/2020,U,145682,76276,221958


In [39]:
#grab 2019 dates
first_df = cta_df[6636:6879]
second_df = cta_df[6953:7074]

#merge dataframes
full_2019 = pd.merge(first_df, second_df, how='outer')

#change service_date column to string for easier manipulation
full_2019 = full_2019.astype({'service_date':'string'})
full_2019[['service_date', 'service_month']] = full_2019.service_date.str.split('/', expand=True)
full_2019

ValueError: Columns must be same length as key

In [38]:
#grab 2020 dates
third_df = cta_df[6879:6952]
fourth_df = cta_df[7075:]

#merge dataframes
full_2020 = pd.merge(third_df, fourth_df, how='outer')
#change service_date column to string for easier manipulation
full_2020 = full_2020.astype({'service_date':'string'})
full_2020.service_date = full_2020.service_date.str.split('/', expand=True)
full_2020

,service_date,day_type,bus,rail_boardings,total_rides
0,01,U,267300,255759,523059
1,01,W,651421,565139,1216560
2,01,W,688165,578202,1266367
3,01,A,409243,306656,715899
4,01,U,314424,234155,548579
...,...,...,...,...,...
329,11,U,119470,56839,176309
330,11,W,221878,106692,328570
331,11,A,198860,97141,296001
332,11,U,145682,76276,221958
